In [44]:
import math
import random

In [47]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, mode='general'):
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter[STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter[STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # ------------------------------------ STA setting ------------------------------------------
    # record STAs choosen RA-RU, sta_on_R[slot_number][STA_1, STA_26...]
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    
    # No.i slot    
    for i in range(0, Imax):
        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < 5):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record access delay slot number
                counter_delay_slot += (i+1)
                success_sta += 1
            # assume always success, and [0] success
            elif (len(sta_on_R[i][r]) == 2):
                # assume always success, and [0] success
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record
                counter_delay_slot += (i+1)
                success_sta += 1
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < 5):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    return success_probability, access_delay, through_put, utilization

In [54]:
sample = 1000

m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
Ps_list_simu = [0 for i in range(20)]
Da_list_simu = [0 for i in range(20)]
T_list_simu = [0 for i in range(20)]
U_list_simu = [0 for i in range(20)]

# m_list_simu = [(i+1)*10 for i in range(50)]
# Ps_list_simu = [0 for i in range(50)]
# Da_list_simu = [0 for i in range(50)]
# T_list_simu = [0 for i in range(50)]
# U_list_simu = [0 for i in range(50)]

R=18

for idx, m in enumerate(m_list_simu):
    success_probability_total = 0;
    access_delay_total = 0;
    through_put_total = 0;
    utilization_total = 0;
    no_success_counter = 0;
    for i in range(sample):
        success_probability, access_delay, through_put, utilization = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, mode='general')
        success_probability_total += success_probability
        access_delay_total += access_delay
        through_put_total += through_put
        utilization_total += utilization
        if success_probability == 0:
            no_success_counter += 1

    Ps = success_probability_total / sample
#     Da = access_delay_total / sample
    Da = access_delay_total / (sample - no_success_counter)
    T = through_put_total / sample
    U = utilization_total / sample
    
    Ps_list_simu[idx] = Ps
    Da_list_simu[idx] = Da
    T_list_simu[idx] = T
    U_list_simu[idx] = U

print(Ps_list_simu)
print(Da_list_simu)
print(T_list_simu)
print(U_list_simu)

[1.0, 1.0, 1.0, 0.9999, 0.9977600000000011, 0.9728666666666699, 0.8724142857142885, 0.7151125000000003, 0.5594333333333339, 0.440290000000001, 0.3463818181818187, 0.2804333333333334, 0.23054615384615312, 0.19114999999999924, 0.16216000000000017, 0.1399437500000001, 0.12232941176470624, 0.10847777777777784, 0.09735789473684206, 0.08726999999999974]
[7.153085700000049, 9.067155900000051, 11.731574899999988, 15.093529251923142, 19.09008126693999, 23.33233703008961, 26.8018308264839, 28.94852545343765, 30.532917745603132, 31.837506526209292, 33.18593160428127, 34.42601004723417, 35.631758514652645, 36.73415162068941, 37.80579065695904, 38.73824398417118, 39.5480863975717, 40.23293414487066, 40.92052939799466, 41.405369307131814]
[1.25, 2.5, 3.75, 4.9995, 6.236, 7.2965, 7.633625, 7.151125, 6.293625, 5.503625, 4.76275, 4.2065, 3.746375, 3.345125, 3.0405, 2.798875, 2.5995, 2.44075, 2.31225, 2.18175]
[0.06944444444444328, 0.13888888888888656, 0.2083333333333358, 0.27774999999999533, 0.34644444